- RealPython tutorial on Redis - https://realpython.com/python-redis/



- [Redis-py documentation](https://redis-py.readthedocs.io/en/stable/)

In [1]:
import redis

In [2]:
r = redis.Redis()

### key: str

In [12]:
r.mset({"Croatia": "Zagreb", "Bahamas": "Nassau"})

True

In [13]:
r.get('Bahamas').decode('utf-8')

'Nassau'

### key: set

In [14]:
import datetime
today = datetime.date.today()
stoday = today.isoformat()
visitors = {"dan", "jon", "alex"}
r.sadd(stoday, *visitors)  # sadd: set-add

0

In [15]:
r.smembers(stoday)

{b'alex', b'dan', b'jon'}

In [10]:
r.scard(stoday)

3

### key: hash

In [34]:
import random

random.seed(444)
hats = {f"hat:{random.getrandbits(32)}": i for i in (
    {
        "color": "black",
        "price": 49.99,
        "style": "fitted",
        "quantity": 1000,
        "npurchased": 0,
    },
    {
        "color": "maroon",
        "price": 59.99,
        "style": "hipster",
        "quantity": 500,
        "npurchased": 0,
    },
    {
        "color": "green",
        "price": 99.99,
        "style": "baseball",
        "quantity": 200,
        "npurchased": 0,
    })
}

In [35]:
with r.pipeline() as pipe:
    for h_id, hat in hats.items():
        print(f"h_id={h_id}")
        pipe.hmset(h_id, hat)
    pipe.execute()

h_id=hat:1326692461
h_id=hat:1236154736
h_id=hat:56854717


In [36]:
r.bgsave()

True

In [37]:
r.hgetall("hat:1326692461")

{b'color': b'black',
 b'price': b'49.99',
 b'style': b'fitted',
 b'quantity': b'1000',
 b'npurchased': b'0'}

In [38]:
r.keys()

[b'Bahamas',
 b'hat:56854717',
 b'hat:1326692461',
 b'Croatia',
 b'hat:1236154736',
 b'2020-05-16']

### buyitem

In [39]:
import logging
import redis

logging.basicConfig()

class OutOfStockError(Exception):
    """Raised when PyHats.com is all out of today's hottest hat"""

def buyitem(r: redis.Redis, itemid: int) -> None:
    with r.pipeline() as pipe:
        error_count = 0
        while True:
            try:
                # Get available inventory, watching for changes
                # related to this itemid before the transaction
                pipe.watch(itemid)
                nleft: bytes = r.hget(itemid, "quantity")
                if nleft > b"0":
                    pipe.multi()
                    pipe.hincrby(itemid, "quantity", -1)
                    pipe.hincrby(itemid, "npurchased", 1)
                    pipe.execute()
                    break
                else:
                    # Stop watching the itemid and raise to break out
                    pipe.unwatch()
                    raise OutOfStockError(
                        f"Sorry, {itemid} is out of stock!"
                    )
            except redis.WatchError:
                # Log total num. of errors by this user to buy this item,
                # then try the same process again of WATCH/HGET/MULTI/EXEC
                error_count += 1
                logging.warning(
                    "WatchError #%d: %s; retrying",
                    error_count, itemid
                )
    return None

In [40]:
hid = 56854717
r.hmget(f"hat:{hid}", "quantity", "npurchased") 


for i in range(4):
    buyitem(r, f"hat:{hid}")
    
r.hmget(f"hat:{hid}", "quantity", "npurchased")     

[b'196', b'4']

#### use json to (de)serialize

In [25]:
import json

In [26]:
with r.pipeline() as pipe:
    for h_id, hat in hats.items():
        print(f"h_id={h_id}")
        pipe.set(h_id, json.dumps(hat))
    pipe.execute()

h_id=hat:1326692461
h_id=hat:1236154736
h_id=hat:56854717


In [27]:
"hat:56854717", json.loads(r.get("hat:56854717"))

('hat:56854717',
 {'color': 'green',
  'price': 99.99,
  'style': 'baseball',
  'quantity': 200,
  'npurchased': 0})

### delete key

In [32]:
for key in r.scan_iter("hat:*"):
    r.delete(key)

In [33]:
r.keys()

[b'Bahamas', b'Croatia', b'2020-05-16']

### TTL - expiry

In [41]:
from datetime import timedelta
# setex: "SET" with expiration
r.setex(
    "runner",
    timedelta(minutes=1),
    value="now you see me, now you don't"
)

True

In [46]:
r.ttl("runner"), r.pttl("runner")  # Like ttl, but milliseconds

(-2, -2)

In [43]:
r.ttl("runner"), r.pttl("runner")  # Like ttl, but milliseconds

(46, 46022)

In [44]:
r.get("runner")

b"now you see me, now you don't"

In [47]:
r.exists("runner")  # Key & value are both gone (expired)

0

### list

In [48]:
r.lpush("ips", "51.218.112.236")
r.lpush("ips", "90.213.45.98")
r.lpush("ips", "115.215.230.176")
r.lpush("ips", "51.218.112.236")

4

In [52]:
r.llen("ips")

4

In [56]:
r.lpush("ips", "127.0.0.1")
r.rpush("ips",  "8.8.8.8")

6

In [57]:
for i in range(r.llen("ips")):
    print(f'i={i},  v={r.lindex("ips", i)}')

i=0,  v=b'127.0.0.1'
i=1,  v=b'51.218.112.236'
i=2,  v=b'115.215.230.176'
i=3,  v=b'90.213.45.98'
i=4,  v=b'51.218.112.236'
i=5,  v=b'8.8.8.8'


In [58]:
ip = r.rpop("ips")
print(ip)

b'8.8.8.8'


In [51]:
r.keys()

[b'Bahamas',
 b'ips',
 b'hat:56854717',
 b'hat:1326692461',
 b'Croatia',
 b'hat:1236154736',
 b'2020-05-16']

### Redis Task Queue

- Flask By Example - https://github.com/realpython/flask-by-example

- Redis Task Queue - https://realpython.com/flask-by-example-implementing-a-redis-task-queue/

